Another test we ran was done by using generalized linear models (GLMs). When doing prelimary research on the topic of differential expressions we found an R package called DESeq2. This package has over 27000 citations and it uses GLMs on the raw count data, along with more technical processes to get differentially expressed genes.  Therefore, since it seems that GLMs are a well preceded method in the bioinformatics community we decided to try our own GLM model to test the research questions.

As for the specifics, we ran a GLM on the original dataset with a gaussian family and canonical link. For the formula we use gene_expression ~ \[variable we are looking at\] + lab + chip.version + constant. The lab and chip version were added to control for any confounding they might have since we are still using the original dataset. Therefore, this method can also be used as a sanity check for making sure our normalization works as desired. If the GLM model and our model that uses our normalized data output similar genes, then we have more confidence in both our results and techniques.

For the males and females split, we ran a GLM on each gene and found the p-value for the sex variable. We then ran the Bonferroni family-wise error rate correction with $\alpha = .05$ and got 2 genes that are significant: DDX3Y and RPS4Y1.

For the region split, we ran a GLM on each gene and found the p-value for the region variable. We then ran the Bonferroni family-wise error rate correction with $\alpha = .05$ and got 4 genes that are significant: CABP1, CARTPT, SCN1B, COX7A1.

We choose to use a Bonferroni correction due to its conservative nature. Since GLMs are well precedented and the Bonferroni procedure is a very conserative correction, we can be very confident that the findings we get for our GLMs are worth looking into.

We also ran a Monte-Carlo permutation test on the difference of means on our research questions using the normalized data. Given a gene, if under the null hypothesis males and females (or A.C.C. vs D.L.P.F.C.) have no differences, then if we aggregate the male and female data points and go through all possible permutations of male/female (A.C.C./D.L.P.F.C.) assignment, then we can see how unlikely our found difference in means was. However, since we have many genes and many samples for each gene, going through all possible permutations is computationally expensive. Thus, we only run 1000 random permutations and get our p-value from these permutations. It is known that the standard deviation of this is $\sqrt{\frac{\hat{p}(1-\hat{p})}{1000}} \leq \sqrt{\frac{.5\cdot.5}{1000}} \approx .0158$, which we deemed acceptable due to the leniency we have been given to find differentably expressed genes.

When running our Monte-Carlo permutation tests for male vs. female, using the Benjamini-Hochberg false discovery rate correction with $\alpha = .1$, we got the following 5 differentiably expressed genes: UTY, USP9Y, KDM5D, DDX3Y, RPS4Y1.

When running our Monte-Carlo permutation tests for A.C. cortex vs. D.L.P.F. cortex, using the Benjamini-Hochberg false discovery rate correction with $\alpha = .1$, we got the following 10 differentiably expressed genes: ZNF609, DAPK3, CARTPT, AP1S1, ZYX, NDUFS8, CAMTA1, CCDC106, SLC9A3R2, RHBDD3. While these only share 1 common gene with the GLM model, we will note that the others declared significant in the GLM model were still among the genes with the smallest $\hat{p}$s, with the largest of them having a $\hat{p}$ being $.015$.

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadr
import statsmodels.formula.api as smf
import statsmodels.api as sm
from tqdm import tqdm
from multiprocess import Pool
from statsmodels.stats.multitest import fdrcorrection, multipletests

## Reading in Data

In [60]:
rda = pyreadr.read_r('data/brain.rda')
expression = rda["expression"]
genes = rda["genes"]
samples = rda["samples"]

norm_data = pd.read_csv("data/normalized_genes.csv")
no_bact_cols = expression.loc[:,[not x for x in genes["chrom"].isna().to_numpy()]].columns

## GLM

### Male vs. Female

In [7]:
glm_no_normal = expression.copy()
glm_no_normal["sex"] = samples["sex"]
glm_no_normal["region"] = samples["region"]
glm_no_normal["lab"] = samples["lab"]
glm_no_normal["chip.version"] = samples["chip.version"]
glm_no_normal["patient"] = samples["patient"]
formula = "38355_at~sex+region+lab+chip.version"
columns = ["sex","lab","chip.version"]
dummies = pd.get_dummies(data=glm_no_normal[columns], columns=columns, drop_first=True)
dummies = sm.add_constant(dummies)
dummies
pvals_m_f = []
#a = smf.glm(formula=formula, data=glm_no_normal, family=sm.families.Gamma(sm.families.links.log())).fit()
for i in no_bact_cols:
    a = sm.GLM(endog=glm_no_normal[i], exog=dummies, family=sm.families.Gaussian()).fit()#sm.families.NegativeBinomial(sm.families.links.log())).fit()
    pvals_m_f.append((a.pvalues[1],i))

In [8]:
for x in np.array(pvals_m_f)[multipletests([x[0] for x in pvals_m_f], alpha=.05, method="bonferroni")[0]]:
    print(genes.loc[x[1]]["sym"])

DDX3Y
RPS4Y1


### A.C.C. vs. D.L.P.F.C.

In [10]:
glm_region = expression.copy()
glm_region["sex"] = samples["sex"]
glm_region["region"] = samples["region"]
glm_region["lab"] = samples["lab"]
glm_region["chip.version"] = samples["chip.version"]
glm_region["patient"] = samples["patient"]
glm_region=glm_region.loc[[x in ["A.C. cortex", "D.L.P.F. cortex"] for x in glm_region["region"]]]
columns = ["region","lab","chip.version"]
dummies = pd.get_dummies(data=glm_region[columns], columns=columns, drop_first=True)
dummies = sm.add_constant(dummies)
dummies = dummies.drop(["region_cerebellum"], axis=1)
pvals_region = []
#a = smf.glm(formula=formula, data=glm_no_normal, family=sm.families.Gamma(sm.families.links.log())).fit()
for i in no_bact_cols:
    a = sm.GLM(endog=glm_region[i], exog=dummies, family=sm.families.Gaussian()).fit()#sm.families.NegativeBinomial(sm.families.links.log())).fit()
    pvals_region.append((a.pvalues[1],i))

In [61]:
for x in np.array(pvals_region)[multipletests([x[0] for x in pvals_region], alpha=.05, method="bonferroni")[0]]:
    print(genes.loc[x[1]]["sym"])

CABP1
CARTPT
SCN1B
COX7A1


## Permutation Testing

In [43]:
def perm(data1, data2, runs, test_stat):
    '''Does a Monte Carlo permutation test'''
    count = 0
    for i in range(runs):
        total_data = np.concatenate((data1, data2))
        perm_data1 = np.random.choice(total_data, len(data1))
        perm_data2 = np.setdiff1d(total_data, perm_data1)
        if test_stat(perm_data1, perm_data2) > test_stat(data1, data2):
            count += 1
    p_hat = count/runs
    return (p_hat, np.sqrt((p_hat*(1-p_hat))/runs)) #p_hat and std_dev returned

def perm_multi(a):
    '''Does a Monte Carlo permutation test, but written so that we can use multiprocessing'''
    data1, data2, runs, test_stat, col = a
    count = 0
    for i in range(runs):
        total_data = np.concatenate((data1, data2))
        perm_data1 = np.random.choice(total_data, len(data1))
        perm_data2 = np.setdiff1d(total_data, perm_data1)
        if test_stat(perm_data1, perm_data2) > test_stat(data1, data2):
            count += 1
    p_hat = count/runs
    return (p_hat, np.sqrt((p_hat*(1-p_hat))/runs), col) #p_hat, std_dev, and column name returned

### Male vs. Female

In [63]:
males = norm_data.loc[norm_data["rownames"].isin(((samples["sex"]=="male").index[(samples["sex"]=="male")]))]
females = norm_data.loc[norm_data["rownames"].isin(((samples["sex"]=="female").index[(samples["sex"]=="female")]))]
columns = norm_data.columns.to_numpy()[1:]

In [44]:
max_pool = 8
arrs = [(males[col], females[col],1000, lambda x,y: abs(np.mean(x) - np.mean(y)), col) for col in columns]
with Pool(max_pool) as p:
    sex_outputs = list(tqdm(p.imap(perm_multi,arrs),total=len(arrs)))

100%|███████████████████████████████████████| 8783/8783 [03:50<00:00, 38.13it/s]


In [52]:
sex_outputs.sort(key=lambda x: x[0])
for x in np.array(sex_outputs)[multipletests([x[0] for x in sex_outputs], alpha=.1, method="fdr_bh")[0]]:
    print(x[2])

UTY
USP9Y
KDM5D
DDX3Y
RPS4Y1


### Region

In [55]:
acc = norm_data.loc[norm_data["rownames"].isin(((samples["region"]=="A.C. cortex").index[(samples["region"]=="A.C. cortex")]))]
dlpfc = norm_data.loc[norm_data["rownames"].isin(((samples["region"]=="D.L.P.F. cortex").index[(samples["region"]=="D.L.P.F. cortex")]))]

In [56]:
max_pool = 8
arrs = [(acc[col], dlpfc[col],1000, lambda x,y: abs(np.mean(x) - np.mean(y)), col) for col in columns]
with Pool(max_pool) as p:
    region_outputs = list(tqdm(p.imap(perm_multi,arrs),total=len(arrs)))

100%|███████████████████████████████████████| 8783/8783 [03:44<00:00, 39.10it/s]


In [64]:
region_outputs.sort(key=lambda x: x[0])
for x in np.array(region_outputs)[multipletests([x[0] for x in region_outputs], alpha=.1, method="fdr_bh")[0]]:
    print(x[2])

ZNF609
DAPK3
CARTPT
AP1S1
ZYX
NDUFS8
CAMTA1
CCDC106
SLC9A3R2
RHBDD3
